In [70]:
!pip install gurobipy

In [71]:
# Load all required libraries
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.utils import resample  # for bootstrapping
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize


## Optimization Project 3

Abhay Puri

Abhiroop Kumar

Ethan Davenport

Liam Thompson

In [72]:
#import data
df = pd.read_csv('price_demand_data.csv')

## Part 1

In [73]:
X = df[["price"]].to_numpy()
y = df["demand"].to_numpy()

ols = LinearRegression()
ols.fit(X, y)

a = float(ols.intercept_)
b = float(ols.coef_[0])
r2 = float(ols.score(X, y))

residuals = y - (a + b*X[:,0])

print(f"intercept: {a:.6f}")
print(f"slope: {b:.6f}")
print(f"r squared: {r2:.6f}")

intercept: 1924.717544
slope: -1367.712524
r squared: 0.621472


## Part 2

In [74]:
c = 0.5
g = 0.75
t = 0.15
p_fixed = 1.0

d1 = a + b*p_fixed + residuals # demand vector at p=1
n = d1.size
print("Scenarios at p=1:", n, "points")

Scenarios at p=1: 99 points


## Part 3

In [75]:
#model
m = gp.Model()
q = m.addMVar(1, lb=0.0, name="q")

s = m.addMVar(n, lb=0.0, name="s") # sales
r = m.addMVar(n, lb=0.0, name="r") # rush print
d = m.addMVar(n, lb=0.0, name="d") # disposal

In [76]:
#constraints
m.addConstr(s <= q[0])
m.addConstr(s <= d1)
m.addConstr(r >= d1 - q[0])
m.addConstr(d >= q[0] - d1)

<MConstr (99,) *awaiting model update*>

In [77]:
#objective and solve
obj = (p_fixed * gp.quicksum(s) - c * n * q[0] - g * gp.quicksum(r) - t * gp.quicksum(d)) / n
m.setObjective(obj, GRB.MAXIMIZE)
m.Params.OutputFlag = 0
m.optimize()

q_star = float(q.X[0])
profit_p1 = float(m.objVal)

print(f"optimal quantity at p=1: {q_star:.6f}")
print(f"Expected profit at p=1: {profit_p1:.6f}")

optimal quantity at p=1: 628.542505
Expected profit at p=1: 178.994225


## Part 4

In [78]:
#model
mq = gp.Model()

p = mq.addMVar(1, lb=0.0, name="p") # price
q = mq.addMVar(1, lb=0.0, name="q") # quantity

s = mq.addMVar(n, lb=0.0, name="s") # sales
r = mq.addMVar(n, lb=0.0, name="r") # rush
d = mq.addMVar(n, lb=0.0, name="d") # disposal

In [79]:
#constraints
d2 = a + b * p[0] + residuals # demand vector under decision p
mq.addConstr(s <= q[0])
mq.addConstr(s <= d2)
mq.addConstr(r >= d2 - q[0])
mq.addConstr(d >= q[0] - d2)

<MConstr (99,) *awaiting model update*>

In [80]:
#objective and solve
obj_qp = (p[0] * gp.quicksum(s) - c * n * q[0] - g * gp.quicksum(r) - t * gp.quicksum(d)) / n
mq.setObjective(obj_qp, GRB.MAXIMIZE)
mq.Params.OutputFlag = 0
mq.optimize()

p_star = float(p.X[0])
q_star_joint = float(q.X[0])
profit_joint = float(mq.objVal)

In [81]:
print(f"optimal price: {p_star:.6f}")
print(f"optimal quantity: {q_star_joint:.6f}")
print(f"Expected profit: {profit_joint:.6f}")

optimal price: 0.943164
optimal quantity: 704.183311
Expected profit: 183.407058
